In [ ]:
-- Margen del producto vs margen promedio global (subconsulta en SELECT)
SELECT p.producto_id, p.nombre, (p.precio_unitario - p.costo_unitario) AS margen,
       (SELECT AVG(precio_unitario - costo_unitario) FROM dbo.dim_productos) AS margen_promedio
FROM dbo.dim_productos p;

In [ ]:
-- Clientes con unidades vendidas > promedio general (subconsulta correlacionada)
SELECT c.cliente_id, c.nombre,
       (SELECT SUM(v.cantidad) FROM dbo.fact_ventas v WHERE v.cliente_id = c.cliente_id) AS unidades_cliente
FROM dbo.dim_clientes c
WHERE (SELECT SUM(v2.cantidad) FROM dbo.fact_ventas v2 WHERE v2.cliente_id = c.cliente_id) >
      (SELECT AVG(sum_cantidad) FROM (SELECT cliente_id, SUM(cantidad) AS sum_cantidad FROM dbo.fact_ventas GROUP BY cliente_id) x);

🟢 Ejercicio: Subconsulta en WHERE para productos con ingreso total > ingreso promedio por producto.

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
WHERE (SELECT SUM(v.cantidad) FROM dbo.fact_ventas v WHERE v.producto_id = p.producto_id) >
      (SELECT AVG(sum_ing) FROM (SELECT producto_id, SUM(cantidad) AS sum_ing FROM dbo.fact_ventas GROUP BY producto_id) y);

🟠 Ejercicio: Usar una subconsulta para calcular número de clientes activos (ventas >0).

In [ ]:
SELECT COUNT(*) AS clientes_activos
FROM dbo.dim_clientes c
WHERE EXISTS (SELECT 1 FROM dbo.fact_ventas v WHERE v.cliente_id = c.cliente_id);

🔴 Reto: Reescribir la consulta de clientes > promedio usando una CTE para mejorar legibilidad.

In [ ]:
WITH ventas_por_cliente AS (
    SELECT cliente_id, SUM(cantidad) AS unidades
    FROM dbo.fact_ventas
    GROUP BY cliente_id
), promedio AS (SELECT AVG(unidades) AS unidades_promedio FROM ventas_por_cliente)
SELECT c.cliente_id, c.nombre, v.unidades
FROM dbo.dim_clientes c
JOIN ventas_por_cliente v ON c.cliente_id = v.cliente_id
CROSS JOIN promedio p
WHERE v.unidades > p.unidades_promedio;

Errores comunes: subconsultas redundantes; no reutilizar lógica con CTE; escalar mal el rendimiento.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.